In [1]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk import ngrams
import random
import codecs
import re
import glob
import string
import io
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC,NuSVC
from nltk.classify import ClassifierI
from statistics import mode

In [2]:
pos_comments = open("F:\\DOCUMENTS\\Python\\myfiles\\NLTK\\positive.txt",encoding='utf-8').read()
neg_comments = open("F:\\DOCUMENTS\\Python\\myfiles\\NLTK\\negative.txt",encoding='utf-8').read()

In [3]:
pos_comments_list = list(pos_comments.split('\n'))
neg_comments_list = list(neg_comments.split('\n'))


Feature Extraction for Uni-Gram and Bi-Gram


In [4]:
def create_word_features_unigram(words):
    unigram_dict = dict((word,True) for word in words.split('\u0020'))     
    return unigram_dict


def create_word_features_bi_gram(words):
    raw = []
    bi_gram = []
    i=0
    
    for w in words.split('\u0020'):
        raw.append(w)

    while i<(len(raw)-1):
        bi_gram.append(raw[i] + " "+raw[i+1])
        i+=1
     
    bigram_dict = dict((word,True) for word in bi_gram)
    return bigram_dict


In [5]:
pos_word_features_uni = []
neg_word_features_uni = []    
pos_word_features_bi = []
neg_word_features_bi = []    


for i in pos_comments_list:
    pos_word_features_uni.append((create_word_features_unigram(i),"positive"))
    pos_word_features_bi.append((create_word_features_bi_gram(i),"positive"))
    
for i in neg_comments_list:
    neg_word_features_uni.append((create_word_features_unigram(i),"negative"))
    neg_word_features_bi.append((create_word_features_bi_gram(i),"negative"))


In [6]:
all_word_features_uni = pos_word_features_uni + neg_word_features_uni
random.shuffle(all_word_features_uni)
trainning_set_uni = all_word_features_uni[:200]
test_set_uni = all_word_features_uni[200:290]

all_word_features_bi = pos_word_features_bi + neg_word_features_bi
random.shuffle(all_word_features_bi)
trainning_set_bi = all_word_features_bi[:200]
test_set_bi = all_word_features_bi[200:290]

# print(" ",len(all_word_features_uni))
# print(" ",len(all_word_features_bi))
# print(" ",len(pos_word_features_uni),len(neg_word_features_uni))
# print(" ",len(pos_word_features_bi) ,len(neg_word_features_bi))
# print(pos_word_features_bi[155])



Naive Bayes Accuracy check for Uni-Gram, Bi-Gram



In [19]:
classifier_train_uni = NaiveBayesClassifier.train(trainning_set_uni)
accuracy_nb_uni = nltk.classify.util.accuracy(classifier_train_uni,test_set_uni)
print("NaiveBayes classifier UniGram: ", (accuracy_nb_uni * 100))
classifier_train_uni.show_most_informative_features(5)

classifier_train_bi = NaiveBayesClassifier.train(trainning_set_bi)
accuracy_nb_bi = nltk.classify.util.accuracy(classifier_train_bi,test_set_bi)
print("\nNaiveBayes classifier Bi-Gram : ", (accuracy_nb_bi * 100))
classifier_train_bi.show_most_informative_features(5)

NaiveBayes classifier UniGram:  71.11111111111111
Most Informative Features
                  ආදරෙයි = True           positi : negati =      7.0 : 1.0
                     රටේ = True           negati : positi =      6.0 : 1.0
                  ලංකාවේ = True           negati : positi =      6.0 : 1.0
                    අපිට = True           negati : positi =      5.2 : 1.0
                     අනේ = True           negati : positi =      4.4 : 1.0

NaiveBayes classifier Bi-Gram :  61.111111111111114
Most Informative Features
            ක්‍රිකට් වලට = True           negati : positi =      2.6 : 1.0
               ගොන් ආතල් = True           negati : positi =      2.6 : 1.0
             ක්‍රීඩා කරන = True           negati : positi =      1.9 : 1.0
                  මේ වගේ = True           negati : positi =      1.9 : 1.0
          කුසල් මෙන්ඩිස් = True           negati : positi =      1.9 : 1.0


In [20]:
LogisticRegression_classifier_uni = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier_uni.train(trainning_set_uni)
accuracy_logistic_uni = nltk.classify.accuracy(LogisticRegression_classifier_uni,test_set_uni)
print("Logistic Regression classifier Uni Gram: ",accuracy_logistic_uni*100)

LogisticRegression_classifier_bi = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier_bi.train(trainning_set_bi)
accuracy_logistic_bi = nltk.classify.accuracy(LogisticRegression_classifier_bi,test_set_bi)
print("Logistic Regression classifier Bi-Gram: ",accuracy_logistic_bi*100)

Logistic Regression classifier Uni Gram:  75.55555555555556
Logistic Regression classifier Bi-Gram:  57.77777777777777


In [21]:
LinearSVC_classifier_uni = SklearnClassifier(LinearSVC())
LinearSVC_classifier_uni.train(trainning_set_uni)
accuracy_svm_uni = nltk.classify.accuracy(LinearSVC_classifier_uni,test_set_uni)
print("SVM classifier Uni Gram: ",accuracy_svm_uni*100)

LinearSVC_classifier_bi = SklearnClassifier(LinearSVC())
LinearSVC_classifier_bi.train(trainning_set_bi)
accuracy_svm_bi = nltk.classify.accuracy(LinearSVC_classifier_bi,test_set_bi)
print("SVM classifier Algo Bi-Gram: ",accuracy_svm_bi*100)

SVM classifier Uni Gram:  71.11111111111111
SVM classifier Algo Bi-Gram:  57.77777777777777




Testing for Positive,Negative comments in Unigram



In [10]:
Test_comment_1 = '''අපි ඔබ සමඟ සිටිමු'''
test_com_1 = []
for r in Test_comment_1.split('\u0020'):
    test_com_1.append(r)
test_dict_1 = dict([(word,True) for word in test_com_1 ])
print(Test_comment_1 +" (Unigram): "+ classifier_train_uni.classify(test_dict_1))


test_comment_2 = '''අපිට දැන් අප්පිරියයි'''
testing_2 = []
for r in test_comment_2.split('\u0020'):
    testing_2.append(r)
test_dict_2 = dict([(word,True) for word in testing_2 ])
print(test_comment_2 +" (Unigram): "+ classifier_train_uni.classify(test_dict_2))

අපි ඔබ සමඟ සිටිමු (Unigram): positive
අපිට දැන් අප්පිරියයි (Unigram): negative


In [11]:
Test_comment_1 = '''අපි ඔබ සමඟ සිටිමු'''
count = 0
test_com_1 = []
for r in Test_comment_1.split('\u0020'):
    test_com_1.append(r)
bi_grammed_1 = []

while count<(len(test_com_1)-1):
        bi_grammed_1.append(test_com_1[count] + " "+test_com_1[count+1])
        count+=1
test_dict_1 = dict([(word,True) for word in bi_grammed_1 ])
print (Test_comment_1 +" (Bi-Gram) : "+classifier_train_bi.classify(test_dict_1))


test_comment_2 = '''අපිට දැන් අප්පිරියයි'''
count = 0
test_com_2 = []
for r in test_comment_2.split('\u0020'):
    test_com_2.append(r)    
bi_grammed_2 = []
while count<(len(test_com_2)-1):
        bi_grammed_2.append(test_com_2[count] + " "+test_com_2[count+1])
        count+=1
test_dict_2 = dict([(word,True) for word in bi_grammed_2 ])
print (test_comment_2 +" (Bi-Gram) : "+classifier_train_bi.classify(test_dict_2))

අපි ඔබ සමඟ සිටිමු (Bi-Gram) : positive
අපිට දැන් අප්පිරියයි (Bi-Gram) : positive


In [17]:
bihara_dict = {
    'Title': 'Shrungaaraya   Mahinda My3 Ranil ;-)',
    'View_Count': '2738691', 
    'Likes': '5350',
    'DisLikes': '1733', 
    'VideoID': 'uJRG1512ygc', 
    'Comment': ['අපි ඔබ සමඟ සිටිමු','අපිට දැන් අප්පිරියයි'],
    '_id': '5d9ad961112acd12bffdf83d'
}

comments = bihara_dict.get('Comment')
print(len(comments))


2


In [31]:
pos_comment_count = 0
neg_comment_count = 0
for i in comments:
    comment = i
    splitted_comments = []
    for r in comment.split('\u0020'):
        splitted_comments.append(r)
    comments_dictionary = dict([(word,True) for word in splitted_comments ])
    vote_classifier_output = Voting_classifer(classifier_train_uni.classify(comments_dictionary),
                                   classifier_train_bi.classify(comments_dictionary),
                                   LogisticRegression_classifier_uni.classify(comments_dictionary),
                                   LogisticRegression_classifier_bi.classify(comments_dictionary),
                                   LinearSVC_classifier_uni.classify(comments_dictionary),
                                   LinearSVC_classifier_bi.classify(comments_dictionary),
                                   accuracy_nb_uni,accuracy_nb_bi,
                                   accuracy_logistic_uni, accuracy_logistic_bi,
                                   accuracy_svm_uni, accuracy_svm_bi
                                  )

    
    if (vote_classifier_output) == 'negative':
        neg_comment_count+=1
    elif (vote_classifier_output) == 'positive':
        pos_comment_count+=1
        
print (pos_comment_count) 
print (neg_comment_count) 

අපි ඔබ සමඟ සිටිමු positive
අපිට දැන් අප්පිරියයි negative
1
1


In [ ]:
def Stop_word_remover(comment):
    

In [36]:
def Voting_classifer(NBclassifer_Uni,NBclassifer_Bi,LR_Uni,LR_Bi,SVM_Uni,SVM_Bi,acc_nb_uni,acc_nb_bi,acc_lr_uni,acc_lr_bi,
                    acc_svm_uni,acc_svm_bi):
    
    positive_votes = 0
    negative_votes = 0
    
    classifier_mode = [NBclassifer_Uni,NBclassifer_Bi,LR_Uni,LR_Bi,SVM_Uni,SVM_Bi]
    
    if NBclassifer_Uni == 'positive': positive_votes +=acc_nb_uni 
    else: negative_votes +=acc_nb_uni      
    if NBclassifer_Bi == 'positive':positive_votes +=acc_nb_bi
    else:negative_votes +=acc_nb_bi      
    if LR_Uni == 'positive': positive_votes +=acc_lr_uni
    else:  negative_votes +=acc_lr_uni       
    if LR_Bi == 'positive':  positive_votes +=acc_lr_bi
    else:  negative_votes +=acc_lr_bi         
    if SVM_Uni == 'positive':positive_votes +=acc_svm_uni
    else: negative_votes +=acc_svm_uni    
    if SVM_Bi == 'positive': positive_votes +=acc_svm_bi
    else: negative_votes +=acc_svm_bi     
    if positive_votes>negative_votes: return ('positive')
    elif positive_votes<negative_votes:return ('negative')
    else:  return (mode(classifier_mode))
    